Configuration

In [2]:
# Đường dẫn thư mục ảnh
IMAGE_FOLDER = "images_png"

# Ngưỡng similarity
SIMILARITY_THRESHOLD = 0.9

# File cache
CACHE_FILE = "feature_cache.pkl"

# File output
OUTPUT_FILE = "duplicates.txt"

In [27]:
import os
import numpy as np
import pandas as pd
import cv2 
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import pickle
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity


Feature Extractor (Trích xuất đặc trưng)

In [29]:
class FeatureExtractor:
    def __init__(self):
        self.model = ResNet50(weights="imagenet", include_top=False, pooling="avg")
    
    def extract_features(self, img_path, target_size=(224, 224)):
        """Trích xuất vector đặc trưng từ một ảnh"""
        try:
            img = image.load_img(img_path, target_size=target_size)
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            features = self.model.predict(x, verbose=0)
            return features[0]
        except Exception as e:
            print(f"Lỗi khi xử lý ảnh {img_path}: {str(e)}")
            return None
    
    def build_feature_database(self, base_folder, cache_file="feature_cache.pkl"):
        """Xây dựng database đặc trưng từ thư mục ảnh"""
        # Kiểm tra cache
        if os.path.exists(cache_file):
            print("Đang tải database từ cache...")
            with open(cache_file, 'rb') as f:
                return pickle.load(f)
        
        feature_db = {}
        valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff')
        
        # Lấy danh sách tất cả file ảnh
        image_files = []
        for root, _, files in os.walk(base_folder):
            for file in files:
                if file.lower().endswith(valid_extensions):
                    image_files.append(os.path.join(root, file))
        
        # Trích xuất đặc trưng với progress bar
        for img_path in tqdm(image_files, desc="Trích xuất đặc trưng"):
            features = self.extract_features(img_path)
            if features is not None:
                feature_db[img_path] = features
        
        # Lưu cache
        with open(cache_file, 'wb') as f:
            pickle.dump(feature_db, f)
        
        return feature_db

Similarity Calculator (Tính toán độ tương đồng)


In [30]:
class SimilarityCalculator:
    @staticmethod
    def cosine_similarity(vec1, vec2):
        """Tính độ tương đồng cosine giữa hai vector"""
        return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))
    
    @staticmethod
    def find_duplicates(feature_db, threshold=0.9):
        """Tìm các cặp ảnh trùng lặp dựa trên ngưỡng similarity"""
        paths = list(feature_db.keys())
        features = np.array([feature_db[path] for path in paths])
        
        # Tính ma trận similarity một lần
        similarity_matrix = cosine_similarity(features)
        
        # Tìm các cặp trùng lặp
        duplicates = []
        n = len(paths)
        
        for i in range(n):
            for j in range(i+1, n):
                if similarity_matrix[i, j] >= threshold:
                    duplicates.append((paths[i], paths[j], similarity_matrix[i, j]))
        
        return duplicates

Results Handler (Xử lý và hiển thị kết quả)

In [34]:
class ResultsHandler:
    @staticmethod
    def print_results(duplicates, threshold=0.9):
        """In kết quả các cặp ảnh trùng lặp kèm label"""
        if not duplicates:
            print("Không tìm thấy ảnh trùng lặp nào.")
            return
        
        print(f"\nTìm thấy {len(duplicates)} cặp ảnh trùng lặp:")
        for img1, img2, sim in duplicates:
            label = 1 if sim >= threshold else 0
            print(f"{os.path.basename(img1)} <--> {os.path.basename(img2)} : similarity = {sim:.4f}, label = {label}")
    
    @staticmethod
    def save_results(duplicates, txt_file="duplicates.txt", excel_file="duplicates.xlsx", threshold=0.9):
        """Lưu kết quả vào file TXT và Excel"""
        if not duplicates:
            with open(txt_file, 'w', encoding='utf-8') as f:
                f.write("Không tìm thấy ảnh trùng lặp nào.")
            print(f"Không tìm thấy ảnh trùng lặp nào. Kết quả đã lưu vào {txt_file}")
            return
        
        # ---- Lưu file TXT ----
        with open(txt_file, 'w', encoding='utf-8') as f:
            f.write(f"Tìm thấy {len(duplicates)} cặp ảnh trùng lặp:\n")
            for img1, img2, sim in duplicates:
                label = 1 if sim >= threshold else 0
                f.write(f"{img1} <--> {img2} : similarity = {sim:.4f}, label = {label}\n")
        
        # ---- Lưu file Excel ----
        data = []
        for img1, img2, sim in duplicates:
            label = 1 if sim >= threshold else 0
            data.append([os.path.basename(img1), os.path.basename(img2), sim, label])
        
        df = pd.DataFrame(data, columns=["Ảnh 1", "Ảnh 2", "Similarity", "Label"])
        df.to_excel(excel_file, index=False)
        
        print(f"\nĐã lưu kết quả vào:\n- {txt_file}\n- {excel_file}")

Main Application

In [35]:
import time


def main():
    start_time = time.time()
    
    # Khởi tạo các thành phần
    extractor = FeatureExtractor()
    calculator = SimilarityCalculator()
    results_handler = ResultsHandler()
    
    # Bước 1: Trích xuất đặc trưng
    print("Bước 1: Đang trích xuất đặc trưng từ ảnh...")
    feature_db = extractor.build_feature_database("images_png")
    
    # Bước 2: Tính toán độ tương đồng
    print("Bước 2: Đang tính toán độ tương đồng...")
    duplicates = calculator.find_duplicates(feature_db, threshold=0.9)
    
    # Bước 3: Hiển thị và lưu kết quả
    print("Bước 3: Đang xử lý kết quả...")
    results_handler.print_results(duplicates)
    results_handler.save_results(duplicates)
    
    end_time = time.time()
    print(f"\nThời gian thực hiện: {end_time - start_time:.2f} giây")

# Trong notebook chỉ cần gọi main()
main()


Bước 1: Đang trích xuất đặc trưng từ ảnh...
Đang tải database từ cache...
Bước 2: Đang tính toán độ tương đồng...
Bước 3: Đang xử lý kết quả...

Tìm thấy 231 cặp ảnh trùng lặp:
106641569.png <--> 57071523.png : similarity = 1.0000, label = 1
131416196.png <--> 74500335.png : similarity = 1.0000, label = 1
154259057.png <--> 263588554.png : similarity = 0.9983, label = 1
174896487.png <--> 85764360.png : similarity = 0.9353, label = 1
194067352.png <--> 212934661.png : similarity = 1.0000, label = 1
194067352.png <--> 274879959.png : similarity = 1.0000, label = 1
194067352.png <--> 277381589.png : similarity = 0.9082, label = 1
194121937.png <--> 194130039.png : similarity = 0.9199, label = 1
212934661.png <--> 274879959.png : similarity = 1.0000, label = 1
212934661.png <--> 277381589.png : similarity = 0.9082, label = 1
222452404.png <--> 274069561.png : similarity = 0.9656, label = 1
262792654.png <--> 269944956.png : similarity = 1.0000, label = 1
262792654.png <--> 270977143.png :